<a href="https://colab.research.google.com/github/gzholtkevych/Design-Compilers-for-DSL/blob/main/SimpleCompiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ПРОСТИЙ ПРИКЛАД КОМПІЛЯТОРА**

## **Імпорт необхідних сутностей**

In [ ]:
from typing import Tuple, Any, Self, List

In [ ]:
from dataclasses import dataclass
import re
from abc import ABC, abstractmethod

## **Арифметичні вирази**

Розглянемо мову арифметичних виразів, що побудовані з

- цілих констанат;
- складених виразів, що мають форму `( operation operand operand )`.

Значеннями для `operation` є `+` або `*`, а значеннями `operand` є арифметисні вирази.

Приклад арифметичного виразу: `(+ (* 3 10) 5)`.

### **Формальний опис синтаксису мови**

Для опису цієї мови ми можемо використати PEG-граматику, яка чітко визначає структуру виразів.

```
Expression      <- Constant / ComposedExpression
Constant        <- '0' / [1-9][0-9]*
ComposedExpression <- '(' Operation Expression Expression ')'
Operation       <- '+' / '*'
```

Пояснення

> `Expression`: Це основне правило, яке каже, що вираз може бути або `Constant`, або `ComposedExpression`.  
`Constant`: Це правило описує цілі числа. Вони можуть бути нулем ('0') або будь-яким числом, що починається з цифри від 1 до 9, за якою можуть іти інші цифри ([1-9][0-9]\*).  
`ComposedExpression`: Це правило описує складений вираз. Він починається з відкритої дужки `'('`, за якою слідує `Operation`, а потім два `Expression` (операнди), і завершується все закритою дужкою `')'`.  
Operation: Це правило визначає, що операція може бути або '+', або '\*'.

### **Лексичні класи та токени**

Лексичними класами (найменшими значущими одиницями) цієї мовиє

1. відкрита дужка `LPAREN`, шаблоном для якої є `r'\('`;
2. закрита дужка `RPAREN`, шаблоном для якої є `r'\)'`;
3. операція `PLUS`, шаблоном для якої є `r'\+'`;
4. операція `MULT`, шаблоном для якої є `r'\*'`;
5. число `NUMBER`, шаблоном для якого є `r'0|[1-9][0-9]*'`.


In [ ]:
LEXICAL_CLASSES = {
    'LPAREN': r'\(',
    'RPAREN': r'\)',
    'PLUS': r'\+',
    'MULT': r'\*',
    'NUMBER': r'0|[1-9][0-9]*'
}

#### **Клас `Token`**

Клас використовується для представлення лексем (найменших смислових одиниць) у лексичному аналізаторі.  
Клас побудований на основі `dataclass` для зручного зберігання даних і має додаткову логіку для валідації та обробки значень.

In [ ]:
@dataclass(frozen=True)
class Token:
    type: str
    value: str

    def __post_init__(self):
        """
        Перевіряє, чи відповідає значення токена його типу,
        використовуючи словник LEXICAL_CLASSES.
        """
        global LEXICAL_CLASSES
        if self.type not in LEXICAL_CLASSES:
            raise ValueError(f"Невідомий тип токена: {self.type}")

        pattern = LEXICAL_CLASSES[self.type]
        if not re.fullmatch(pattern, self.value):
            raise ValueError(
                f"Некоректне значення '{self.value}' для типу токена "
                f"'{self.type}'. Очікувався патерн: {pattern}"
            )


#### **Приклад використання**

1. Коректний токен

In [ ]:
try:
    token_valid = Token(type='NUMBER', value='123')
    print(f"Створено коректний токен: {token_valid}")
except ValueError as e:
    print(e)

Створено коректний токен: Token(type='NUMBER', value='123')


2. Некоректне значення токену

In [ ]:
try:
    token_invalid_value = Token(type='NUMBER', value='abc')
except ValueError as e:
    print(f"Помилка! {e}")

Помилка! Некоректне значення 'abc' для типу токена 'NUMBER'. Очікувався патерн: 0|[1-9][0-9]*


3. Невідомий лексичний клас

In [ ]:
try:
    token_invalid_type = Token(type='UNKNOWN', value='!')
except ValueError as e:
    print(f"Помилка! {e}")

Помилка! Невідомий тип токена: UNKNOWN


#### **Лексичний аналізатор**

Лексичний аналізатор, який перетворює вхідний рядок на список токенів.

In [ ]:
class Lexer:

    def __init__(self, text: str):
        self.text = text
        self.pos = 0

    def get_token(self) -> Token | None:
        """Повертає наступний токен з вхідного рядка."""
        self.skip_whitespace()

        if self.pos >= len(self.text):
            return None # Кінець рядка

        # Пошук першого відповідного патерна
        for token_type, pattern in LEXICAL_CLASSES.items():
            match = re.match(pattern, self.text[self.pos:])
            if match:
                value = match.group(0)
                self.pos += len(value)
                return Token(token_type, value)

        raise ValueError(
            f"Невідомий токен на позиції {self.pos}: {self.text[self.pos:]}")

    def skip_whitespace(self):
        """Пропускає пробіли в рядку."""
        match = re.match(r'\s+', self.text[self.pos:])
        if match:
            self.pos += len(match.group(0))

    def tokenize(self) -> list[Token]:
        """Виконує повний лексичний аналіз і повертає список токенів."""
        tokens = []
        while True:
            token = self.get_token()
            if token is None:
                break
            tokens.append(token)
        return tokens


##### Приклад використання

In [ ]:
text_input = "(+ (* 3 10) 5)"
lexer = Lexer(text_input)
token_list = lexer.tokenize()

print("Лексичний аналіз рядка:", text_input)
print("---")
for token in token_list:
    print(token)

Лексичний аналіз рядка: (+ (* 3 10) 5)
---
Token(type='LPAREN', value='(')
Token(type='PLUS', value='+')
Token(type='LPAREN', value='(')
Token(type='MULT', value='*')
Token(type='NUMBER', value='3')
Token(type='NUMBER', value='10')
Token(type='RPAREN', value=')')
Token(type='NUMBER', value='5')
Token(type='RPAREN', value=')')


### **Парсинг**

#### **Структура даних для AST**

In [ ]:
class Expression:
    pass

@dataclass(frozen=True)
class Constant(Expression):
    value: int

@dataclass(frozen=True)
class Plus(Expression):
    left: Expression
    right: Expression

@dataclass(frozen=True)
class Mult(Expression):
    left: Expression
    right: Expression

Клас `Expression` є базовим класом.  
Він не призначений для створення об'єктів безпосередньо.  
Його мета — забезпечити абстрагування від конкретного виду виразів, що представлені класами, які від нього успадковуються і які представляють конкретні вирази:

1. константи (цілі числа);
2. суми двох інших виразів;
3. добутки двох інших виразів.

#### **Клас `Parser`**

In [ ]:
class Parser:
    def __init__(self, tokens: List[Token]):
        self.tokens = tokens
        self.pos = 0

    def current_token(self) -> Token | None:
        """Повертає поточний токен або None, якщо досягнуто кінця."""
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return None

    def consume(self, expected_type: str) -> Token:
        """Перевіряє тип поточного токена та переходить до наступного."""
        token = self.current_token()
        if token is None or token.type != expected_type:
            raise SyntaxError(
                f"Очікувався токен типу '{expected_type}', але отримано {token}"
            )
        self.pos += 1
        return token

    def parse_constant(self) -> Constant:
        """Парсить константу."""
        token = self.consume('NUMBER')
        return Constant(value=int(token.value))

    def parse_expression(self) -> Expression:
        """Головний метод, що розбирає будь-який вираз."""
        token = self.current_token()
        if token is None:
            raise SyntaxError("Неочікуваний кінець вхідних даних")

        if token.type == 'NUMBER':
            return self.parse_constant()
        elif token.type == 'LPAREN':
            return self.parse_composed_expression()
        else:
            raise SyntaxError(f"Неочікуваний токен: {token}")

    def parse_composed_expression(self) -> Expression:
        """Парсить складений вираз."""
        self.consume('LPAREN') # Споживаємо '('

        token = self.current_token()
        if token.type == 'PLUS':
            self.consume('PLUS')
            left = self.parse_expression()
            right = self.parse_expression()
            self.consume('RPAREN') # Споживаємо ')'
            return Plus(left, right)
        elif token.type == 'MULT':
            self.consume('MULT')
            left = self.parse_expression()
            right = self.parse_expression()
            self.consume('RPAREN') # Споживаємо ')'
            return Mult(left, right)
        else:
            raise SyntaxError(f"Очікувалась операція, але отримано {token}")

    def parse(self) -> Expression:
        """Головний метод, що запускає парсинг."""
        ast = self.parse_expression()
        if self.current_token() is not None:
            raise SyntaxError(f"Залишилися зайві токени після парсингу: "
                              f"{self.current_token()}")
        return ast

##### Приклад використання

Вхідний рядок

In [ ]:
text_input = "(+ (* 3 10) 5)"

1. Лексичний аналіз

In [ ]:
lexer = Lexer(text_input)
tokens = lexer.tokenize()
print("Лексеми:")
for token in tokens:
    print(token)

Лексеми:
Token(type='LPAREN', value='(')
Token(type='PLUS', value='+')
Token(type='LPAREN', value='(')
Token(type='MULT', value='*')
Token(type='NUMBER', value='3')
Token(type='NUMBER', value='10')
Token(type='RPAREN', value=')')
Token(type='NUMBER', value='5')
Token(type='RPAREN', value=')')


2. Синтаксичний аналіз

In [ ]:
print("\nПобудова AST:")
parser = Parser(tokens)
ast_tree = parser.parse()
print(ast_tree)


Побудова AST:
Plus(left=Mult(left=Constant(value=3), right=Constant(value=10)), right=Constant(value=5))


## **Стековий обчислювач**

Простий стековий обчислювач виконує операції, використовуючи стек - список, що працює за принципом LIFO (Last In, First Out).

Підтримуються дві основні команди:
- "save": проштовхує число в стек та
- "eval": виконує операцію ("plus" або "mult") з двома верхніми елементами стека, видаляючи їх та натомість проштовхуючи в стек результат.

### **Клас `StackCalculator`**

Клас `StackCalculator` імітує роботу простого стекового комп'ютера.  

In [ ]:
class StackCalculator:
    """
    Простий стековий обчислювач, що виконує операції, використовуючи стек.
    """
    def __init__(self):
        # Стек як список Python
        self._stack = []

    def save(self, number: int):
        """Проштовхує число на стек."""
        self._stack.append(number)

    def eval(self, operation: str):
        """
        Виконує операцію з двома верхніми елементами стека.
        У випадку недостатньої кількості елементів на стеку, буде викликана
        помилка ValueError.
        """
        if len(self._stack) < 2:
            raise ValueError(
                "Помилка! На стеку недостатньо елементів для виконання "
                "операції 'eval'. Необхідно щонайменше 2 елементи.")
        # Видаляємо два верхні елементи
        operand2 = self._stack.pop()
        operand1 = self._stack.pop()
        result = 0
        if operation == "plus":
            result = operand1 + operand2
        elif operation == "mult":
            result = operand1 * operand2
        else:
            raise ValueError(f"Помилка! Невідома операція: {operation}")
        # Поміщаємо результат назад на стек
        self._stack.append(result)

    def get_result(self) -> int | None:
        """Повертає значення з вершини стека."""
        if not self._stack:
            return None
        return self._stack[-1]

#### Приклад використання

1. Створюємо калкулятор

In [ ]:
calculator = StackCalculator()

2. Демонстрація успішного виконання

In [ ]:
if calculator._stack:
    calculator._stack = []
print("--- Успішне виконання ---")
calculator.save(3)
print(f"Стек після 'save 3': {calculator._stack}") # Стек: [3]
calculator.save(10)
print(f"Стек після 'save 10': {calculator._stack}") # Стек: [3, 10]
calculator.save(5)
print(f"Стек після 'save 5': {calculator._stack}") # Стек: [3, 10, 5]
calculator.eval("plus") # 10 + 5 = 15
print(f"Стек після 'eval plus': {calculator._stack}") # Стек: [3, 15]
calculator.eval("mult") # 3 * 15 = 45
print(f"Стек після 'eval mult': {calculator._stack}") # Стек: [45]
print(f"Результат: {calculator.get_result()}\n")

--- Успішне виконання ---
Стек після 'save 3': [3]
Стек після 'save 10': [3, 10]
Стек після 'save 5': [3, 10, 5]
Стек після 'eval plus': [3, 15]
Стек після 'eval mult': [45]
Результат: 45



3. Демонстрація помилки: недостатньо елементів

In [ ]:
print("--- Демонстрація помилки ---")
calculator_error = StackCalculator()
calculator_error.save(10)
print(f"Стек після 'save 10': {calculator_error._stack}") # Стек: [10]

try:
    calculator_error.eval("mult") # Спроба виконати операцію з одним елементом
except ValueError as e:
    print(f"Виникла помилка: {e}")

--- Демонстрація помилки ---
Стек після 'save 10': [10]
Виникла помилка: Помилка! На стеку недостатньо елементів для виконання операції 'eval'. Необхідно щонайменше 2 елементи.


## **Компіляція**

### **Клас `Translator`**

In [ ]:
# Команда для стекового обчислювача
Command = Tuple[str, int | str | None]

class Translator:
    """
    Транслятор, що перетворює AST на послідовність команд
    для стекового обчислювача.
    """
    def __init__(self):
        self.commands: List[Command] = []

    def translate(self, node: Expression):
        """
        Рекурсивно обходить AST і генерує команди.
        """
        if isinstance(node, Constant):
            # Генеруємо команду save для константи
            self.commands.append(("save", node.value))

        elif isinstance(node, Plus):
            # Спочатку обробляємо лівий та правий операнди
            self.translate(node.left)
            self.translate(node.right)
            # Потім генеруємо команду eval
            self.commands.append(("eval", "plus"))

        elif isinstance(node, Mult):
            # Спочатку обробляємо лівий та правий операнди
            self.translate(node.left)
            self.translate(node.right)
            # Потім генеруємо команду eval
            self.commands.append(("eval", "mult"))

        else:
            raise TypeError(f"Невідомий тип вузла AST: {type(node)}")

    def get_commands(self) -> List[Command]:
        """Повертає згенерований список команд."""
        return self.commands

### **Приклад використання**

In [ ]:
text_input = "(+ (* 3 10) 5)"
print(f"Вхідний рядок: '{text_input}'\n")

print(" ---- Лексичний аналіз ----------- ")
lexer = Lexer(text_input)
tokens = lexer.tokenize()
print("\nПослідовність лексем:\n")
for token in tokens:
    print(token)
print()
print(" ---- Синтаксичний аналіз -------- ")
parser = Parser(tokens)
ast_tree = parser.parse()
print("\nАбстрактне синтаксичне дерево:\n")
print(ast_tree)
print()
print(" ---- Компіляція ----------------- ")
translator = Translator()
translator.translate(ast_tree)
commands = translator.get_commands()
print("\nЗгенеровані команди:\n")
for cmd, arg in commands:
    print(f"- {cmd} {arg}")

Вхідний рядок: '(+ (* 3 10) 5)'

 ---- Лексичний аналіз ----------- 

Послідовність лексем:

Token(type='LPAREN', value='(')
Token(type='PLUS', value='+')
Token(type='LPAREN', value='(')
Token(type='MULT', value='*')
Token(type='NUMBER', value='3')
Token(type='NUMBER', value='10')
Token(type='RPAREN', value=')')
Token(type='NUMBER', value='5')
Token(type='RPAREN', value=')')

 ---- Синтаксичний аналіз -------- 

Абстрактне синтаксичне дерево:

Plus(left=Mult(left=Constant(value=3), right=Constant(value=10)), right=Constant(value=5))

 ---- Компіляція ----------------- 

Згенеровані команди:

- save 3
- save 10
- eval mult
- save 5
- eval plus
